In [1]:
#全局变量
hub_token = open('/root/hub_token.txt').read().strip()
repo_id = 'lansinuote/nlp.2.predict_middle_word'
push_to_hub = True

In [2]:
from transformers import AutoTokenizer

#加载编码器
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base', use_fast=True)

print(tokenizer)

#编码试算
tokenizer.batch_encode_plus([
    'hide new secretions from the parental units',
    'contains no wit , only labored gags'
])

RobertaTokenizerFast(name_or_path='distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})


{'input_ids': [[0, 37265, 92, 3556, 2485, 31, 5, 20536, 2833, 2], [0, 10800, 5069, 117, 22094, 2156, 129, 6348, 3995, 821, 8299, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [3]:
from datasets import load_dataset


def get_dataset():
    #加载数据
    dataset = load_dataset(path='glue', name='sst2')

    #分词,同时删除多余的字段
    def f(data):
        return tokenizer.batch_encode_plus(data['sentence'])

    dataset = dataset.map(f,
                          batched=True,
                          batch_size=1000,
                          num_proc=4,
                          remove_columns=['sentence', 'idx', 'label'])

    #过滤掉太短的句子
    def f(data):
        return [len(i) >= 9 for i in data['input_ids']]

    dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)

    #截断句子,同时整理成模型需要的格式
    def f(data):
        b = len(data['input_ids'])
        data['labels'] = data['attention_mask'].copy()
        for i in range(b):
            #裁剪长度到9
            data['input_ids'][i] = data['input_ids'][i][:9]
            data['attention_mask'][i] = [1] * 9
            data['labels'][i] = [-100] * 9

            #input_ids最后一位是2
            data['input_ids'][i][-1] = 2

            #每一句话第4个词为mask
            #tokenizer.get_vocab()['<mask>'] -> 50264
            data['labels'][i][4] = data['input_ids'][i][4]
            data['input_ids'][i][4] = 50264

        return data

    dataset = dataset.map(f, batched=True, batch_size=1000, num_proc=4)
    
    return dataset


if push_to_hub:
    dataset = get_dataset()
    dataset.push_to_hub(repo_id=repo_id, token=hub_token)

#直接使用我处理好的数据集
dataset = load_dataset(path=repo_id)

dataset, dataset['train'][0]

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-40784360ebdc84b7.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7d3d2f7713a0fe97.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-cfc4ff0ff52a2696.arrow


...

Generating validation split:   0%|          | 0/861 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/44279 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--nlp.2.predict_middle_word-ac370b90ccc7f67b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

(DatasetDict({
     test: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 1776
     })
     validation: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 861
     })
     train: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 44279
     })
 }),
 {'input_ids': [0, 37265, 92, 3556, 50264, 31, 5, 20536, 2],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, -100, -100, -100, 2485, -100, -100, -100, -100]})

In [4]:
import torch
from transformers.data.data_collator import default_data_collator

#能够实现随机mask的collate_fn
#如果要使用这个工具类,在数据预处理时就不需要设置数据中的mask,然后让labels=input_ids.copy即可
#from transformers import DataCollatorForLanguageModeling
#data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm_probability=0.1)

#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=default_data_collator,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

len(loader), data

(5534,
 {'input_ids': tensor([[    0,   102, 35063,  2156, 50264,  7953,    12,  4530,     2],
          [    0,     6,  1195,    87, 50264,   201,     7, 18651,     2],
          [    0, 35685,     4, 28519, 50264,    16, 10686,  1035,     2],
          [    0,   428, 29022,  2329, 50264,  2156,     5,   232,     2],
          [    0,     6, 42327,   924, 50264,   128,    29,   124,     2],
          [    0, 39763,     9,    42, 50264,  6667,    50, 10132,     2],
          [    0,   405, 28289, 18711, 50264,   350,   203,    15,     2],
          [    0,   627,  5128, 38187, 50264,    16,   295,    75,     2]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1]]),
  'labels': tensor([[ -100,  -100,  -100,  -10

In [5]:
from transformers import AutoModelForCausalLM, RobertaModel, PreTrainedModel, PretrainedConfig

#加载模型
#model = AutoModelForCausalLM.from_pretrained('distilroberta-base')


#定义下游任务模型
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.pretrained = RobertaModel.from_pretrained('distilroberta-base')

        decoder = torch.nn.Linear(768, tokenizer.vocab_size)
        decoder.bias = torch.nn.Parameter(torch.zeros(tokenizer.vocab_size))

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(768, 768),
            torch.nn.GELU(),
            torch.nn.LayerNorm(768, eps=1e-5),
            decoder,
        )

        #加载预训练模型的参数
        parameters = AutoModelForCausalLM.from_pretrained('distilroberta-base')
        self.fc[0].load_state_dict(parameters.lm_head.dense.state_dict())
        self.fc[2].load_state_dict(parameters.lm_head.layer_norm.state_dict())
        self.fc[3].load_state_dict(parameters.lm_head.decoder.state_dict())

        self.criterion = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        logits = self.pretrained(input_ids=input_ids,
                                 attention_mask=attention_mask)
        logits = logits.last_hidden_state

        logits = self.fc(logits)

        loss = None
        if labels is not None:
            shifted_logits = logits[:, :-1].reshape(-1, tokenizer.vocab_size)
            shifted_labels = labels[:, 1:].reshape(-1)

            loss = self.criterion(shifted_logits, shifted_labels)

        return {'loss': loss, 'logits': logits}


model = Model(PretrainedConfig())

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

out = model(**data)

out['loss'], out['logits'].shape

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


12136.4313


(tensor(18.7176, grad_fn=<NllLossBackward0>), torch.Size([8, 9, 50265]))

In [6]:
#测试
def test():
    model.eval()

    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=8,
        collate_fn=default_data_collator,
        shuffle=True,
        drop_last=True,
    )

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):
        #保存下数据中的label,后面计算正确率要用
        label = data['labels'][:, 4].clone()

        #从数据中抹除掉label,防止模型作弊
        data['labels'] = None

        #计算
        with torch.no_grad():
            out = model(**data)

        #[8, 10, 50265] -> [8, 10]
        out = out['logits'].argmax(dim=2)[:, 4]

        correct += (label == out).sum().item()
        total += 8

        if i % 10 == 0:
            print(i)
            print(label)
            print(out)

        if i == 50:
            break

    print(correct / total)

    for i in range(8):
        print(tokenizer.decode(data['input_ids'][i]))
        print(tokenizer.decode(label[i]), tokenizer.decode(out[i]))


test()

0
tensor([  475,  4149,  1368, 36114,  3159,    76,  1412,    80])
tensor([10146,  1295,  2265, 14598, 28805,  1485, 10899,    80])
10
tensor([ 543,    9,   10,   10,   12, 1569,  822,  275])
tensor([ 2579,     9,    10, 24282,    12,   898,  7122,  3968])
20
tensor([   47,   963,    16,  1180,    33, 27229,   352,  8444])
tensor([   52,  4864,    21,  1180,   240, 38821,   352, 26760])
30
tensor([  10,   92,   14,  385,   98, 3360,    5,   42])
tensor([  10, 6087,   14, 3553,  182, 9001,    5,   42])
40
tensor([  254, 18408,   350,   254,  3820,     8,  1666,    12])
tensor([ 1245,  6434, 15605,  1073, 14439,  2156,   149,    12])
50
tensor([   11, 35499,    16,   269,   352,     8, 23317, 11235])
tensor([  562,  3374,  3374,   269,  2156,  2432, 41406, 11235])
0.35049019607843135
<s>rambles on<mask> a disj</s>
 in  getting
<s>the film often<mask> a mesmerizing</s>
 achieves  becomes
<s>its underlying mythology<mask> a hodge</s>
 is  becomes
<s>imagine a<mask> bad community theater</s

In [7]:
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    model.to(device)
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            label = data['labels'][:, 4]
            out = out['logits'].argmax(dim=2)[:, 4]

            correct = (label == out).sum().item()
            accuracy = correct / 8

            lr = optimizer.state_dict()['param_groups'][0]['lr']

            print(i, loss.item(), accuracy, lr)

    model.to('cpu')


if push_to_hub:
    train()
    model.push_to_hub(repo_id=repo_id, use_auth_token=hub_token)

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 16.206497192382812 0.25 1.9996385977593064e-05
50 5.723601341247559 0.5 1.9815684857246115e-05
100 5.552875518798828 0.25 1.963498373689917e-05
150 4.01291561126709 0.375 1.9454282616552225e-05
200 3.9359703063964844 0.375 1.927358149620528e-05
250 3.3738934993743896 0.375 1.909288037585833e-05
300 4.1954450607299805 0.5 1.8912179255511386e-05
350 2.413621187210083 0.625 1.873147813516444e-05
400 4.649163246154785 0.5 1.8550777014817495e-05
450 2.281928062438965 0.5 1.8370075894470547e-05
500 7.373494625091553 0.0 1.81893747741236e-05
550 2.061326026916504 0.625 1.8008673653776656e-05
600 2.4588379859924316 0.75 1.7827972533429708e-05
650 4.1831464767456055 0.625 1.7647271413082763e-05
700 2.7687013149261475 0.375 1.7466570292735818e-05
750 3.4589526653289795 0.5 1.728586917238887e-05
800 2.0876877307891846 0.5 1.7105168052041924e-05
850 4.327033996582031 0.125 1.692446693169498e-05
900 2.654650926589966 0.5 1.674376581134803e-05
950 1.3270173072814941 0.625 1.6563064691001085e-05
10

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

In [8]:
#直接使用我训练好的模型
model = Model.from_pretrained(repo_id)

test()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


0
tensor([11290,    29,  3251,     9,  5730,  2230,  3360,  2614])
tensor([ 5073,    29,  1569,     9, 12801, 44009,   822,  1478])
10
tensor([ 3599,    10,    84, 14207,    10, 36396, 34501,   473])
tensor([ 4780,    10,    84,  2316,    10,  1569, 16887,   473])
20
tensor([14838,  5494, 10295,  1344,   563,  1144,   213,   144])
tensor([ 6269,  5494, 31790,  1344,   631,  1403,   213,   144])
30
tensor([  492,    34,     7,  4839, 20030,  2156,  2019,  8396])
tensor([  492, 15033,     7,    16,  5282,  2156,  6990,  8396])
40
tensor([ 2156,   475,  2801,    10,  5906, 39913,   631,   224])
tensor([ 2156, 15549,  1581,    10, 34029, 13938,   631, 34600])
50
tensor([  14,   28,    5,  740, 1473, 2084,   16,   75])
tensor([   45,    28,    39,  9931, 29941,  2084,    16,   269])
0.5122549019607843
<s>the truth is<mask> the truth about</s>
 that  not
<s>audiences can<mask> expected to suspend</s>
 be  be
<s>foster nails<mask> role, giving</s>
 the  his
<s>the use of<mask>gi and digital</